# Interactive Universal Hyperbolic Geometry

This notebook provides interactive examples of UHG principles and constructions. You can experiment with different points, transformations, and geometric operations to build intuition for hyperbolic geometry.

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from uhg.projective import ProjectiveUHG

# Initialize UHG
uhg = ProjectiveUHG()

## 1. Points in Hyperbolic Space

Let's start by understanding different types of points in hyperbolic space:

In [ ]:
# Regular point inside disk
regular_point = torch.tensor([0.3, 0.0, 1.0])

# Null point on boundary
null_point = torch.tensor([1.0, 0.0, 1.0])

# Point at infinity
infinity = torch.tensor([1.0, 0.0, 0.0])

print(f"Regular point is null: {uhg.is_null_point(regular_point)}")
print(f"Null point is null: {uhg.is_null_point(null_point)}")
print(f"Infinity point properties: x²+y²-z² = {infinity[0]**2 + infinity[1]**2 - infinity[2]**2}")

## 2. Interactive Midpoint Construction

Let's create an interactive tool to explore midpoint construction:

In [ ]:
def plot_points_with_midpoints(x1, y1, x2, y2):
    # Create points
    A = torch.tensor([x1, y1, 1.0])
    B = torch.tensor([x2, y2, 1.0])
    
    # Calculate midpoints
    m1, m2 = uhg.midpoints(A, B)
    
    # Plot
    fig, ax = plt.subplots(figsize=(8, 8))
    circle = plt.Circle((0, 0), 1, fill=False, color='black')
    ax.add_artist(circle)
    
    # Plot original points
    ax.scatter([x1, x2], [y1, y2], c='blue', label='Original Points')
    
    # Plot midpoints if they exist
    if m1 is not None:
        m1_x = m1[0]/m1[2]
        m1_y = m1[1]/m1[2]
        ax.scatter(m1_x, m1_y, c='red', label='Midpoint 1')
        
    if m2 is not None:
        m2_x = m2[0]/m2[2]
        m2_y = m2[1]/m2[2]
        ax.scatter(m2_x, m2_y, c='green', label='Midpoint 2')
    
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.legend()
    
    # Print properties
    if m1 is not None and m2 is not None:
        print("\nMidpoint Properties:")
        print(f"Quadrance A to m₁: {uhg.quadrance(A, m1):.4f}")
        print(f"Quadrance B to m₁: {uhg.quadrance(B, m1):.4f}")
        print(f"m₁⊥m₂ dot product: {uhg.hyperbolic_dot(m1, m2):.4e}")
        print(f"Cross-ratio: {uhg.cross_ratio(A, B, m1, m2):.4f}")
    
    plt.show()

# Example usage
plot_points_with_midpoints(0.3, 0.0, 0.4, 0.2)

## 3. Projective Transformations

Let's explore how projective transformations affect geometric properties:

In [ ]:
def apply_and_plot_transformation(points, matrix):
    # Transform points
    transformed = torch.stack([uhg.transform(p, matrix) for p in points])
    
    # Plot original and transformed
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    for ax, pts, title in [(ax1, points, 'Original'), (ax2, transformed, 'Transformed')]:
        # Plot points
        x = pts[:, 0] / pts[:, 2]
        y = pts[:, 1] / pts[:, 2]
        ax.scatter(x, y)
        
        # Add unit circle
        circle = plt.Circle((0, 0), 1, fill=False, color='black')
        ax.add_artist(circle)
        
        ax.set_xlim(-1.1, 1.1)
        ax.set_ylim(-1.1, 1.1)
        ax.set_aspect('equal')
        ax.set_title(title)
        ax.grid(True)
    
    plt.show()
    return transformed

# Create some points
points = torch.tensor([
    [0.3, 0.0, 1.0],
    [0.4, 0.2, 1.0],
    [0.0, 0.5, 1.0]
])

# Create and apply transformation
matrix = uhg.get_projective_matrix(2)
transformed = apply_and_plot_transformation(points, matrix)

# Verify invariants
print("\nInvariant Properties:")
print("Original cross-ratio:", uhg.cross_ratio(points[0], points[1], points[2], points[0]).item())
print("Transformed cross-ratio:", uhg.cross_ratio(transformed[0], transformed[1], transformed[2], transformed[0]).item())

## 4. Edge Cases and Numerical Stability

Let's explore how the library handles various edge cases:

In [ ]:
def explore_edge_case(case_name, point1, point2):
    print(f"\n{case_name}:")
    print(f"Point 1: {point1}")
    print(f"Point 2: {point2}")
    
    # Try to calculate midpoints
    m1, m2 = uhg.midpoints(point1, point2)
    
    print("Results:")
    print(f"Midpoints exist: {m1 is not None and m2 is not None}")
    if m1 is not None:
        print(f"First midpoint: {m1}")
        if m2 is not None:
            print(f"Second midpoint: {m2}")
            print(f"Perpendicular: {abs(uhg.hyperbolic_dot(m1, m2)) < 1e-5}")

# Test cases
cases = [
    ("Same point", torch.tensor([0.3, 0.0, 1.0]), torch.tensor([0.3, 0.0, 1.0])),
    ("Null point", torch.tensor([1.0, 0.0, 1.0]), torch.tensor([0.3, 0.0, 1.0])),
    ("Points too far", torch.tensor([0.3, 0.0, 1.0]), torch.tensor([2.0, 0.0, 1.0])),
    ("Nearly coincident", torch.tensor([0.3, 0.0, 1.0]), torch.tensor([0.3 + 1e-6, 0.0, 1.0]))
]

for name, p1, p2 in cases:
    explore_edge_case(name, p1, p2)